In [14]:
# Ensemble learning 
'''여러 분류기를 조합하여 사이킷런 투표 기반 분류기를 만들고 훈련'''

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=5000)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)     
# Soft voting을 하기 위해 클래스 확률을 제공하지 않는 SVC 클래스의 매개변수 probability=True로 수정
    # Soft voting은 모든 VotingClassifier의 분류기가 클래스 확률을 예측할 수 있는 경우 가능하다. 
# Hard voting 하는 경우 그대로 진행해도 됨

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf,),('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [15]:
from sklearn.metrics import accuracy_score  # 각 분류기의 정확도 확인
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.888
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


In [16]:
'''하나의 분류기를 사용하지만, 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 방법'''

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=1)  # n_jobs: 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 지정 
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [17]:
'''oob 평가'''
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.99925

In [18]:
'''oob score에서 얻은 accuracy만큼 test set에서 성능을 보일 것 (oob_Score를 구할 때
sample들이 모두 처음 보는 sample일 것이므로'''
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [19]:
bag_clf.oob_decision_function_

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [20]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [21]:
'''BaggingClassifier를 사용해 RandomForestClassifier와 거의 유사하게 만듬'''
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features='sqrt', max_leaf_nodes=16), n_estimators=500
)

In [22]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.0919307354334444
sepal width (cm) 0.02382490951838265
petal length (cm) 0.42227867781196937
petal width (cm) 0.46196567723620346


In [23]:
'''AdaBoost with 결정 트리 max_depth = 1'''
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm='SAMME.R', learning_rate=0.5
)
ada_clf.fit(X_train, y_train)


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

In [24]:
'''Gradient Boosting'''
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

# 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor 훈련
y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

# 두 번째 예측기에서 생긴 잔여 오차에 세 번째 회귀 모델 훈련
y3 = y2-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

# 세 번 째 트리를 포함하는 앙상블 모델이 생김
# 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 된다
'''y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))'''

'y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))'

In [26]:
from sklearn.ensemble import GradientBoostingRegressor

gbrd = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrd.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [28]:
'''120개 트리로 GBRT 앙상블 훈련시키고 최적의 트리 수를 찾기 위해 각 훈련 단계에서
검증 오차를 측정한다
마지막에 최적의 트리 수를 사용해서 새로운 GBRT 앙상블 훈련'''
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [29]:
'''다섯 번의 반복 동안 검증 오차 향상되지 않으면 훈련 멈춤'''
'''
warm_start=True
사이킷런이 fit()메서드가 호출될 떄 기존 트리를 유지하고 훈련을 추가할 수 있다
'''
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train,y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break   # 조기 종료

In [32]:
'''XGBoost'''

import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

# XGBoost는 자동 조기 종료와 같은 기능 제공
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

ModuleNotFoundError: No module named 'xgboost'